## Resumo aula 3 de Machine Learning 2 ##

Também é importante mencionar que nem todos os algoritmos demandam necessariamente a aplicação de codificação. Árvores de decisão e modelos em árvore, em geral, conseguem trabalhar com atributos codificados em textos.

O *Label Encoding* é a técnica mais simples de se realizar codificação de variáveis categóricas. Essencialmente funciona atribuindo simplesmente os valores numéricos para cada valor único da categoria encontrada

<img src="https://miro.medium.com/max/772/1*QQe-4476Oy3_dI1vhb3dDg.png">

O Label encoder não considera a ordem da classificação.

O Ordinal encoder considera a ordem.

In [121]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error, recall_score,f1_score,classification_report, precision_score, accuracy_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

### Exercício 1

Agora vamos utilizar o pipeline e column-transformer num problema de regressão. Com a base 'Steel_industry_data.csv':

- carregue a base
- Elimine as seguintes colunas: `Date`
- Aplique um column transformer nas colunas WeekStatus, Day_of_Week, Load_type
- Eliminar dados nulos
- Separar 30 % para teste
- Use o Random Search com 5 iterações para selecionar os hiperparâmetros (**Cuidado** com o número de hiperparâmetros pois o conjunto de dados é grande)
- Use um objeto `KFold` para validação cruzada (use 3 folds)
- As métricas de seleção podem ser o $R^2$ ou o MAE
- Teste os modelos `Lasso`, e `RandomForest`
- Utilize o modelo do `Pipeline`
- Não se esqueça de aplicar o escalonamento dos dados (`StandardScaler`)

In [122]:
steel = pd.read_csv('Steel_industry_data.csv')

steel.head()

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [123]:
X = steel.drop(columns=['date','Usage_kWh'], axis=1)
y = steel['Usage_kWh']


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

ct = ColumnTransformer([('categorical', OneHotEncoder(drop='first'), ['WeekStatus', 'Day_of_week', 'Load_Type'])],
                       remainder='passthrough')

kf3 = KFold(n_splits=3, shuffle=True)

steps = [('ct', ct),
         ('scaler', StandardScaler()),
         ('rfr', RandomForestRegressor())]

param_dict = {
    'rfr__n_estimators': np.random.randint(1, 20, size = 5),
    'rfr__max_depth': [2,3,4,5]
}
pipe = Pipeline(steps=steps)

model = RandomizedSearchCV(estimator=pipe, param_distributions=param_dict, n_jobs=-1, scoring='r2',cv=kf3, return_train_score=True)

model.fit(X_train,y_train)
model.best_params_

{'rfr__n_estimators': 6, 'rfr__max_depth': 5}

In [124]:
model.fit(X_train,y_train)

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
                   estimator=Pipeline(steps=[('ct',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('categorical',
                                                                               OneHotEncoder(drop='first'),
                                                                               ['WeekStatus',
                                                                                'Day_of_week',
                                                                                'Load_Type'])])),
                                             ('scaler', StandardScaler()),
                                             ('rfr', RandomForestRegressor())]),
                   n_jobs=-1,
                   param_distributions={'rfr__max_depth': [2, 3, 4, 5],
                                      

In [125]:
model.predict(X_test)

array([ 3.1535735 ,  4.16832387, 46.86294424, ...,  4.16832387,
       63.90450875, 61.39647639])

In [126]:
model.best_score_

0.9889356301982294